In [ ]:
!pip install validators

In [ ]:
import nltk
from nltk.corpus import twitter_samples,stopwords 
from nltk.tokenize import word_tokenize 
import string
import validators
import numpy as np
from nltk.stem import WordNetLemmatizer 
nltk.download('wordnet') 
nltk.download('twitter_samples')
nltk.download('stopwords')
WL = WordNetLemmatizer() 

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')
tweets = all_positive_tweets + all_negative_tweets
labels = np.append(np.ones((len(all_positive_tweets),1)), np.zeros((len(all_negative_tweets),1)), axis = 0)
print(labels.shape)

(10000, 1)


In [ ]:
def Process_Tweet(tweet):
  tweet = tweet.replace("n't"," not").replace("'m"," am").replace("'ve"," have").replace("’","'").replace('`','')
  tweet = tweet.split()
  alphabets = "qwertyuioplkjhgfdsazxcvbnm"
  stop_words = set(stopwords.words('english'))
  tweet = [w.lower() for w in tweet if w not in stop_words and w[0] not in string.punctuation and not validators.url(w) and w[0] in alphabets]
  tweet = word_tokenize(" ".join(tweet))
  tweet = ["<s>"]+[WL.lemmatize(w) for w in tweet if w not in string.punctuation] +["<\s>"]
  out = []
  return tweet

In [ ]:
from sklearn.model_selection import train_test_split
processsed_i = []
X = []
Y = []
vocab = set()
for i in range(len(tweets)):
  processsed_i = Process_Tweet(tweets[i])
  print("\rTweets",str(i+1),":",processsed_i,end="")
  if processsed_i != []:
    vocab.update(processsed_i)
    if labels[i]==1:
      Y.append(1)
    else:
      Y.append(0)
    X.append(processsed_i)
print("\nNumber of tweets:", len(X),len(Y))
vocab = list(vocab)
print("\nVocab Size:",len(vocab))
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,shuffle=True)

Tweets 10000 : ['<s>', 'supporter', 'expecting', 'misserable', 'week', '<\\s>']
Number of tweets: 10000 10000

Vocab Size: 7996


In [ ]:
vocab_list = []
j = 0
for i in vocab:
  print("\r",j,"/",len(vocab),end = "")
  vocab_list.append(i)
  j+=1
print("\r",len(vocab_list))

 7996


In [ ]:
vocab_dict = {}
counter = 0
for i in list(vocab):
  vocab_dict[i] = False
  print("\rCounter:",counter,end = "")
  counter+=1

Counter: 7995

In [46]:
from nltk import classify
from nltk import NaiveBayesClassifier
train_data = []
test_data = []
print(len(y_train))
for i,j in zip(x_train,y_train):
  ins_dict = vocab_dict.copy()
  for w in i:
    if w in ins_dict.keys():
      ins_dict[w] = True
  train_data.append((ins_dict,j))
for i,j in zip(x_test,y_test):
  ins_dict = vocab_dict.copy()
  for w in i:
    if w in ins_dict.keys():
      ins_dict[w] = True
  test_data.append((ins_dict,j))
print(train_data[:10])

We will get an accuracy of nearly 70%, which while decent is not that great. The reason for low accuracy is 

8000
[({'shipping': False, 'dedication': False, 'landscape': False, 'entry': False, 'seed': False, 'hearing': False, 'comfy': False, 'many': False, 'pocket': False, 'throwback': False, 'barking': False, 'outside': False, 'ff': False, 'virgin': False, 'prod': False, 'x15.319': False, 'aboard': False, 'freaking': False, 'oodles': False, 'ew': False, 'muscle': False, 'creamist': False, 'wobbly': False, 'wailay': False, 'gyu': False, 'maaaaaay': False, 'bob': False, 'pfb': False, 'jaysus': False, 'mango': False, 'nosebleed': False, 'mi': False, 'giving': False, 'takeaway': False, 'hypercholesteloremia': False, 'blowing': False, 'zumba': False, '..tenerina': False, 'black': False, 'louis': False, 'tweaked': False, 'lonely': False, 'x6': False, 'ranking': False, 'consistency': False, 'else': False, 'purity': False, 'favour': False, 'no-no': False, 'soloist': False, 'xxxxxx': False, 'cherish': False, 'tunaita': False, 'robe': False, 'data': False, 'grown': False, 'basketball': False, 'netherl

In [47]:
classifier = NaiveBayesClassifier.train(train_data)
print("Accuracy is:", classify.accuracy(classifier, test_data))
print(classifier.show_most_informative_features(10))

Most Informative Features
                follower = True                1 : 0      =     43.8 : 1.0
                     sad = True                0 : 1      =     33.3 : 1.0
                 arrived = True                1 : 0      =     21.8 : 1.0
               community = True                1 : 0      =     19.0 : 1.0
                    miss = True                0 : 1      =     16.4 : 1.0
                    blog = True                1 : 0      =     13.7 : 1.0
                 welcome = True                1 : 0      =     13.6 : 1.0
                     via = True                1 : 0      =     12.8 : 1.0
                   great = True                1 : 0      =     12.7 : 1.0
                  thanks = True                1 : 0      =     12.4 : 1.0
None
